In [ ]:
# !pip install sentence-transformers pandas scikit-learn torch

In [ ]:
# !pip install huggingface_hub

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
# !pip install -U sentence_transformers

# 2. 데이터셋 로드 및 분할 (Train,Val,Test)

In [ ]:
import json
import torch
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation

# 1. 모델 및 데이터 로드 (경로는 여욱님 환경에 맞춰 확인해 주세요)
model_id = "google/embeddinggemma-300m"
model = SentenceTransformer(model_id)

def load_json_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 데이터 경로 다시 한 번 체크!
train_data = load_json_data('./data/train_dataset.json')
val_data = load_json_data('./data/val_dataset.json')

# 2. 학습용 데이터 객체(InputExample) 생성
train_examples = [
    InputExample(texts=[item['anchor'], item['positive'], item['negative']])
    for item in train_data
]

# 3. 데이터 로더 및 손실 함수 설정
# Batch Size는 여욱님 PC 사양에 맞춰 16~32 사이로 조절하세요.
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.MultipleNegativesRankingLoss(model)

# 4. 검증기(Evaluator) 설정
# 학습 중간에 모델의 실력이 얼마나 늘었는지 체크하는 도구입니다.
queries = {str(i): item['anchor'] for i, item in enumerate(val_data)}
corpus = {str(i): item['positive'] for i, item in enumerate(val_data)}
relevant_docs = {str(i): {str(i)} for i, item in enumerate(val_data)}

evaluator = evaluation.InformationRetrievalEvaluator(
    queries, corpus, relevant_docs, name="card-val-task"
)

# 5. 학습 실행 (model.fit 방식 - 모든 버전에서 호환됨)
print("🚀 [안전 모드] Gemma-300M 학습을 시작합니다...")

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=3,                  # 전체 데이터를 3번 훑습니다.
    warmup_steps=100,          # 초기 안정화 단계
    output_path="./models/gemma-300m-finetuned-card",
    evaluation_steps=200,      # 200번의 걸음마다 시험을 봅니다.
    save_best_model=True,
    show_progress_bar=True
)

print("✨ 모든 과정이 끝났습니다! ./models/gemma-300m-finetuned-card 폴더를 확인하세요.")

In [6]:
import json
import torch
import os
import gc
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation
from tqdm.autonotebook import tqdm

# 1. GPU 메모리 초기화 (이전 작업 찌꺼기 제거)
gc.collect()
torch.cuda.empty_cache()

# 2. 모델 로드 및 그래디언트 체크포인팅 활성화
# 이 기능은 연산 속도를 약간 희생하는 대신 VRAM 사용량을 대폭 줄여줍니다.
MODEL_ID = "google/embeddinggemma-300m"
OUTPUT_PATH = "./models/gemma-300m-4080super-extreme"
model = SentenceTransformer(MODEL_ID)
model[0].auto_model.gradient_checkpointing_enable()

# 3. Gemma 공식 지시어 포맷 함수
def format_query(text):
    return f"task: search result | query: {text}"

def format_doc(content, title="none"):
    return f"title: {title} | text: {content}"

def load_json_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

# 데이터 로드
train_data = load_json_data('./data/train_dataset.json')
val_data = load_json_data('./data/val_dataset.json')

# 4. 학습 데이터셋 구성 (Prefix 적용)
train_examples = []
for item in train_data:
    card_title = item.get('metadata', {}).get('card_name', '카드혜택')
    train_examples.append(InputExample(texts=[
        format_query(item['anchor']), 
        format_doc(item['positive'], title=card_title),
        format_doc(item['negative'], title=card_title)
    ]))

# 5. 4080 Super 익스트림 배치 설정
# AMP + Gradient Checkpointing 조합으로 256 도전에 나섭니다.
BATCH_SIZE = 128 
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=BATCH_SIZE)
train_loss = losses.MultipleNegativesRankingLoss(model)

# 6. 검증기 설정
val_queries = {str(i): format_query(item['anchor']) for i, item in enumerate(val_data)}
val_corpus = {str(i): format_doc(item['positive']) for i, item in enumerate(val_data)}
val_relevant_docs = {str(i): {str(i)} for i, item in enumerate(val_data)}

evaluator = evaluation.InformationRetrievalEvaluator(
    val_queries, val_corpus, val_relevant_docs, name="card-val-task"
)

# 7. 진행 상황 실시간 모니터링 콜백
def training_callback(score, epoch, steps):
    print(f"\n📊 [REPORT] Step: {steps} | Epoch: {epoch} | Accuracy@1: {score:.4f}")

# 8. 최적화 학습 실행
print(f"🔥 [Extreme Mode] RTX 4080 Super 가동! (Batch: {BATCH_SIZE})")
print(f"🛠️ 활성화 기술: AMP(혼합정밀도) + Gradient Checkpointing")

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=10,
    warmup_steps=100,
    optimizer_params={'lr': 2e-5},
    output_path=OUTPUT_PATH,
    evaluation_steps=20,     # 배치가 크므로 20스텝마다 점수 확인 (진행 상황 체크)
    save_best_model=True,
    show_progress_bar=True,
    use_amp=True,            # 혼합 정밀도 활성화
    callback=training_callback # 실시간 점수 출력
)

print(f"✨ 모든 과정 완료! 최적의 모델은 {OUTPUT_PATH}에 있습니다.")

🔥 [Extreme Mode] RTX 4080 Super 가동! (Batch: 128)
🛠️ 활성화 기술: AMP(혼합정밀도) + Gradient Checkpointing


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss,Card-val-task Cosine Accuracy@1,Card-val-task Cosine Accuracy@3,Card-val-task Cosine Accuracy@5,Card-val-task Cosine Accuracy@10,Card-val-task Cosine Precision@1,Card-val-task Cosine Precision@3,Card-val-task Cosine Precision@5,Card-val-task Cosine Precision@10,Card-val-task Cosine Recall@1,Card-val-task Cosine Recall@3,Card-val-task Cosine Recall@5,Card-val-task Cosine Recall@10,Card-val-task Cosine Ndcg@10,Card-val-task Cosine Mrr@10,Card-val-task Cosine Map@100
20,No log,No log,0.603871,0.767742,0.832258,0.913548,0.603871,0.255914,0.166452,0.091355,0.603871,0.767742,0.832258,0.913548,0.752219,0.701285,0.705925
40,No log,No log,0.680000,0.829677,0.896774,0.958710,0.680000,0.276559,0.179355,0.095871,0.680000,0.829677,0.896774,0.958710,0.815290,0.769704,0.772001
49,No log,No log,0.669677,0.833548,0.899355,0.958710,0.669677,0.277849,0.179871,0.095871,0.669677,0.833548,0.899355,0.958710,0.811697,0.764829,0.767191
60,No log,No log,0.668387,0.832258,0.892903,0.962581,0.668387,0.277419,0.178581,0.096258,0.668387,0.832258,0.892903,0.962581,0.813547,0.766013,0.768089
80,No log,No log,0.677419,0.843871,0.883871,0.958710,0.677419,0.281290,0.176774,0.095871,0.677419,0.843871,0.883871,0.958710,0.817749,0.772779,0.775222
98,No log,No log,0.676129,0.849032,0.907097,0.961290,0.676129,0.283011,0.181419,0.096129,0.676129,0.849032,0.907097,0.961290,0.818403,0.772653,0.775015
100,No log,No log,0.681290,0.850323,0.910968,0.966452,0.681290,0.283441,0.182194,0.096645,0.681290,0.850323,0.910968,0.966452,0.823616,0.777909,0.779802
120,No log,No log,0.667097,0.843871,0.904516,0.966452,0.667097,0.281290,0.180903,0.096645,0.667097,0.843871,0.904516,0.966452,0.817409,0.769590,0.771451
140,No log,No log,0.676129,0.840000,0.899355,0.962581,0.676129,0.280000,0.179871,0.096258,0.676129,0.840000,0.899355,0.962581,0.817619,0.771388,0.773507
147,No log,No log,0.669677,0.847742,0.899355,0.960000,0.669677,0.282581,0.179871,0.096000,0.669677,0.847742,0.899355,0.960000,0.817036,0.771040,0.773590



📊 [REPORT] Step: 20 | Epoch: 0.40816326530612246 | Accuracy@1: 0.7522

📊 [REPORT] Step: 40 | Epoch: 0.8163265306122449 | Accuracy@1: 0.8153

📊 [REPORT] Step: 49 | Epoch: 1.0 | Accuracy@1: 0.8117

📊 [REPORT] Step: 60 | Epoch: 1.2244897959183674 | Accuracy@1: 0.8135

📊 [REPORT] Step: 80 | Epoch: 1.6326530612244898 | Accuracy@1: 0.8177

📊 [REPORT] Step: 98 | Epoch: 2.0 | Accuracy@1: 0.8184

📊 [REPORT] Step: 100 | Epoch: 2.0408163265306123 | Accuracy@1: 0.8236

📊 [REPORT] Step: 120 | Epoch: 2.4489795918367347 | Accuracy@1: 0.8174

📊 [REPORT] Step: 140 | Epoch: 2.857142857142857 | Accuracy@1: 0.8176

📊 [REPORT] Step: 147 | Epoch: 3.0 | Accuracy@1: 0.8170

📊 [REPORT] Step: 160 | Epoch: 3.2653061224489797 | Accuracy@1: 0.8183

📊 [REPORT] Step: 180 | Epoch: 3.673469387755102 | Accuracy@1: 0.8298

📊 [REPORT] Step: 196 | Epoch: 4.0 | Accuracy@1: 0.8313

📊 [REPORT] Step: 200 | Epoch: 4.081632653061225 | Accuracy@1: 0.8323

📊 [REPORT] Step: 220 | Epoch: 4.489795918367347 | Accuracy@1: 0.8306

📊 [

In [7]:
import json
import os
from sentence_transformers import SentenceTransformer, evaluation

# 1. 모델 로드
FINAL_MODEL_PATH = "./models/gemma-300m-4080super-extreme"
final_model = SentenceTransformer(FINAL_MODEL_PATH)

# 2. 데이터 로드
def load_json_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

test_data = load_json_data('./data/test_dataset.json')

# 3. Evaluator 설정
test_queries = {str(i): item['anchor'] for i, item in enumerate(test_data)}
test_corpus = {str(i): item['positive'] for i, item in enumerate(test_data)}
test_relevant_docs = {str(i): {str(i)} for i, item in enumerate(test_data)}

# 평가 이름(name)을 기준으로 결과 딕셔너리의 키 값이 생성됩니다.
TASK_NAME = "card-test-task"
test_evaluator = evaluation.InformationRetrievalEvaluator(
    test_queries, test_corpus, test_relevant_docs, name=TASK_NAME
)

# 4. 최종 평가 실행
print("🔍 테스트 데이터를 이용한 심층 성능 분석을 시작합니다...")
results = test_evaluator(final_model)

# 5. [수정] 모든 상세 지표 출력 로직
def print_detailed_report(res, name):
    # 우리가 보고 싶은 지표 리스트 (코사인 유사도 기준)
    metrics = [
        ("Accuracy@1 (Hit Rate)", f"{name}_cosine_accuracy@1"),
        ("Accuracy@10 (Hit Rate)", f"{name}_cosine_accuracy@10"),
        ("MRR@10", f"{name}_cosine_mrr@10"),
        ("NDCG@10", f"{name}_cosine_ndcg@10"),
        ("Precision@10", f"{name}_cosine_precision@10"),
        ("Recall@10", f"{name}_cosine_recall@10"),
    ]
    
    print(f"\n{'='*25} [Gemma-300M 최종 검색 성적표] {'='*25}")
    for label, key in metrics:
        val = res.get(key, 0)
        print(f"✅ {label:<30} : {val:.4f}")
    print(f"{'='*78}\n")

print_detailed_report(results, TASK_NAME)

The tokenizer you are loading from './models/gemma-300m-4080super-extreme' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


🔍 테스트 데이터를 이용한 심층 성능 분석을 시작합니다...

========================= [Gemma-300M 최종 검색 성적표] =========================
✅ Accuracy@1 (Hit Rate)          : 0.7384
✅ Accuracy@10 (Hit Rate)         : 0.9613
✅ MRR@10                         : 0.8191
✅ NDCG@10                        : 0.8540
✅ Precision@10                   : 0.0961
✅ Recall@10                      : 0.9613



# 3. 모델 정의 및 손실함수 설정

- Google의 최신 embeddinggemma-300m 모델을 로드합니다.
- 질문(Anchor)과 정답(Positive)은 가깝게, 오답(Negative)은 멀게 만드는 Triplet Loss를 사용하며, 코사인 유사도 기준 마진을 0.5로 설정합니다.

# 4. 모델 파인튜닝 (Fine-tuning) 실행

- 본격적인 학습을 진행합니다. 학습 속도 향상을 위해 use_amp=True를 사용하며, 5 에포크 동안 검증 점수가 가장 높은 모델을 자동으로 저장합니다.

In [ ]:
# !pip install chromadb

In [8]:
import json
import os
import torch
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

# ==========================================
# 1. 환경 설정 및 모델 로드
# ==========================================
# 데이터 경로 및 모델 경로 설정
INPUT_FILE = './data/FINAL_MASTER_DATA_FIXED_7757.json'      # 7,757건의 카드 원본 데이터
MODEL_PATH = './models/gemma-300m-4080super-extreme'          # 여욱님이 학습시킨 Gemma 임베딩 모델
CHROMA_DB_PATH = './data/chroma_db'                         # 벡터 데이터가 저장될 폴더 경로

# GPU 사용 가능 여부 확인 (속도 향상을 위해 GPU 권장)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"📦 모델을 로드 중입니다... (Device: {device})")

# SentenceTransformer를 통해 Gemma 모델을 메모리에 올립니다.
model = SentenceTransformer(MODEL_PATH, device=device)

# ==========================================
# 2. ChromaDB 커스텀 임베딩 함수 정의
# ==========================================
# ChromaDB가 스스로 임베딩을 하지 않고, 우리가 만든 Gemma 모델을 사용하도록 다리를 놓아주는 클래스입니다.
class GemmaEmbeddingFunction(embedding_functions.EmbeddingFunction):
    def __call__(self, input_texts):
        # [중요] 인덱싱(저장)할 때는 모델 학습 시 사용한 'title: none | text: ' 형식을 지켜야 검색 성능이 나옵니다.
        passages = [f"title: none | text: {text}" for text in input_texts]
        embeddings = model.encode(passages, convert_to_tensor=False)
        return embeddings.tolist()  # DB 저장을 위해 리스트 형태로 반환

# ==========================================
# 3. ChromaDB 초기화 및 데이터 인덱싱
# ==========================================
# 데이터를 하드디스크에 영구 저장하는 클라이언트를 생성합니다.
client = chromadb.PersistentClient(path=CHROMA_DB_PATH)
embedding_fn = GemmaEmbeddingFunction()

# 'card_benefits'라는 이름의 데이터 저장 공간(컬렉션)을 만듭니다.
# metadata={"hnsw:space": "cosine"}는 벡터 간의 거리를 '코사인 유사도'로 계산하겠다는 설정입니다.
collection = client.get_or_create_collection(
    name="card_benefits",
    embedding_function=embedding_fn,
    metadata={"hnsw:space": "cosine"} 
)

# DB에 데이터가 하나도 없다면 최초 1회 인덱싱을 수행합니다.
if collection.count() == 0:
    print("🚀 ChromaDB에 최초 데이터 인덱싱을 시작합니다...")
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        master_data = json.load(f)
    
    # 7,757건의 데이터를 DB 형식에 맞게 분리합니다.
    documents = [item['embedding_input'] for item in master_data]   # 검색 대상 텍스트
    metadatas = [item['metadata'] for item in master_data]          # 카드 이름, 회사 등 부가 정보
    
    # 나중에 검색 결과에서 바로 요약을 보여주기 위해 메타데이터에 미리 넣어둡니다.
    for i, m in enumerate(metadatas):
        m['summary'] = master_data[i]['ai_structured']['summary']
    
    # 각 데이터의 고유 ID 생성 (card_0, card_1...)
    ids = [f"card_{i}" for i in range(len(master_data))]

    # [중요] 한꺼번에 넣으면 메모리가 터질 수 있으므로 500개씩 나눠서 저장합니다.
    batch_size = 500
    for i in range(0, len(documents), batch_size):
        collection.add(
            documents=documents[i:i+batch_size],
            metadatas=metadatas[i:i+batch_size],
            ids=ids[i:i+batch_size]
        )
    print(f"✅ 총 {collection.count()}건의 데이터 저장 완료!")
else:
    print(f"🔄 기존 ChromaDB를 로드했습니다. (저장된 데이터: {collection.count()}건)")

# ==========================================
# 4. 핵심 함수: 다중 의도 교차 검색 (AND 로직)
# ==========================================
def find_combined_card_chroma(query_list, top_k=5, search_depth=100):
    """
    query_list: ['의도1', '의도2'] 검색 키워드 리스트
    top_k: 최종적으로 몇 개를 보여줄 것인가
    search_depth: 각 의도별로 상위 몇 개까지 훑어볼 것인가 (깊을수록 정확함)
    """
    intent_hits = []

    # --- 1단계: 각 의도별 개별 검색 ---
    for sub_query in query_list:
        # 질문용 공식 프롬프트 적용 (학습 시 instruction과 일치시켜야 함)
        query_text = f"task: search result | query: {sub_query}"
        
        # DB에서 유사한 카드 search_depth만큼 추출
        results = collection.query(
            query_texts=[query_text],
            n_results=search_depth,
            include=['metadatas', 'distances']
        )
        
        current_hits = {}
        for i in range(len(results['ids'][0])):
            meta = results['metadatas'][0][i]
            name = meta['card_name']
            
            # [유사도 계산] 거리가 0이면 점수는 1점(만점), 거리가 1이면 점수는 0점
            score = 1 - results['distances'][0][i]
            
            # 같은 카드가 중복 검색될 경우 가장 점수가 높은 것만 기록
            if name not in current_hits:
                current_hits[name] = {
                    "score": score,
                    "benefit": meta.get('summary', '혜택 정보 없음'),
                    "corp": meta['corp']
                }
        intent_hits.append(current_hits)

    # --- 2단계: 교집합(Intersection) 찾기 ---
    # 모든 의도 리스트에 이름이 공통적으로 존재하는 카드만 골라냅니다.
    common_names = set(intent_hits[0].keys()) # 첫 번째 의도 결과들
    for hits in intent_hits[1:]:
        common_names &= set(hits.keys())      # 이후 의도들과 계속 겹치는 것만 남김

    # --- 3단계: 결과 병합 및 점수 합산 ---
    final_recommendations = []
    for name in common_names:
        # 각 의도에서 받은 유사도 점수를 모두 더합니다 (종합 점수)
        total_score = sum(hits[name]['score'] for hits in intent_hits)
        # 각 의도 검색 시 발견된 혜택 요약들을 중복 없이 수집합니다.
        all_benefits = list(set([hits[name]['benefit'] for hits in intent_hits]))
        corp = intent_hits[0][name]['corp']
        
        final_recommendations.append({
            "name": name,
            "corp": corp,
            "score": total_score,
            "benefits": all_benefits
        })

    # --- 4단계: 최종 정렬 및 출력 ---
    # 종합 점수(score)가 높은 순서대로 상위 top_k개 선정
    final_recommendations = sorted(final_recommendations, key=lambda x: x['score'], reverse=True)[:top_k]

    print(f"\n🔍 분석된 의도: {query_list}")
    print(f"✅ 교집합 필터링 결과: {len(common_names)}건 발견")
    print("="*85)
    
    for i, res in enumerate(final_recommendations):
        # 최종 결과 출력
        print(f"{i+1}. [{res['corp']}] {res['name']} (종합 점수: {res['score']:.4f})")
        for b in res['benefits']:
            print(f"     - {b}")
        print("-" * 85)

# ==========================================
# 5. 실행 테스트
# ==========================================
# 사용자의 복합적인 질문을 두 개의 핵심 의도로 나누어 입력합니다.
intents = [
    "배달의민족 요기요 쿠팡이츠 배달 음식 할인 혜택", 
    "버스 지하철 택시 대중교통 교통비 할인 혜택"
]

# 함수 호출
find_combined_card_chroma(intents)

📦 모델을 로드 중입니다... (Device: cuda)


The tokenizer you are loading from './models/gemma-300m-4080super-extreme' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
C:\Users\Jeon\AppData\Local\Temp\ipykernel_21324\2110197300.py:39: DeprecationWarning: The class GemmaEmbeddingFunction does not implement __init__. This will be required in a future version.
  embedding_fn = GemmaEmbeddingFunction()


🔄 기존 ChromaDB를 로드했습니다. (저장된 데이터: 7757건)

🔍 분석된 의도: ['배달의민족 요기요 쿠팡이츠 배달 음식 할인 혜택', '버스 지하철 택시 대중교통 교통비 할인 혜택']
✅ 교집합 필터링 결과: 6건 발견
1. [롯데카드] LOCA for Coffee (종합 점수: 0.8735)
     - 배달의민족과 요기요에서 10% 할인 혜택을 제공합니다.
     - 버스, 지하철, 택시 이용 시 10% 할인 혜택 제공.
-------------------------------------------------------------------------------------
2. [하나카드] 원더카드 2.0 DAILY (종합 점수: 0.8692)
     - 대중교통 이용 시 10% 할인 혜택 제공
     - 딜리버리 서비스에서 10% 할인을 제공하는 카드 혜택입니다.
-------------------------------------------------------------------------------------
3. [우리카드] NU Uniq (종합 점수: 0.8655)
     - 대중교통, 택시, 전기차, 주유에서 1.5% 할인 혜택 제공
     - 배달의민족, B마트, 쿠팡이츠에서 1.5% 할인 혜택 제공
-------------------------------------------------------------------------------------
4. [삼성카드] CU·배달의민족 삼성카드 taptap (종합 점수: 0.8454)
     - CU에서 1,500원당 200원 결제일 할인 혜택 제공.
     - 대중교통 이용 시 1,000원당 100원 결제일 할인 혜택 제공.
-------------------------------------------------------------------------------------
5. [KB국민카드] 토스 USS NEXT 체크카드 (종합 점수: 0.8416)
     -

In [ ]:
# %pip install -qU cohere langchain_cohere

In [ ]:
import json
import os
import torch
import chromadb
import cohere
from dotenv import load_dotenv
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from chromadb.utils import embedding_functions

# 1. 환경 변수 로드 (API 키)
load_dotenv()
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
COHERE_KEY = os.getenv("COHERE_API_KEY")

# 2. 로컬 경로 및 설정
INPUT_FILE = './data/FINAL_MASTER_DATA_FIXED_7757.json'
MODEL_PATH = './models/gemma-300m-4080super-extreme'    
CHROMA_DB_PATH = './data/chroma_db'

# 3. 모델 및 클라이언트 초기화
device = "cuda" if torch.cuda.is_available() else "cpu"
bi_encoder = SentenceTransformer(MODEL_PATH, device=device)
openai_client = OpenAI(api_key=OPENAI_KEY)
co = cohere.ClientV2(api_key=COHERE_KEY)

# 4. 데이터 리하이드레이션 맵 생성 (ID 기반 복원)
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    master_data = json.load(f)

card_master_map = {}
for item in master_data:
    c_id = item['metadata']['card_id']
    if c_id not in card_master_map:
        card_master_map[c_id] = {
            "name": item['metadata']['card_name'],
            "corp": item['metadata']['corp'],
            "full_details": [item['content']], 
            "structured": item['ai_structured']
        }
    else:
        card_master_map[c_id]["full_details"].append(item['content'])

# 5. ChromaDB 연결 및 임베딩 함수 설정
class GemmaEmbeddingFunction(embedding_functions.EmbeddingFunction):
    def __call__(self, input_texts):
        passages = [f"title: none | text: {text}" for text in input_texts]
        return bi_encoder.encode(passages, convert_to_tensor=False).tolist()

chroma_client = chromadb.PersistentClient(path=CHROMA_DB_PATH)
collection = chroma_client.get_collection(
    name="card_benefits", 
    embedding_function=GemmaEmbeddingFunction()
)

# ==========================================
# [HELPER] 동적 의도 추출 함수
# ==========================================
def extract_search_intents(user_query):
    """사용자 질문에서 검색용 핵심 키워드를 동적으로 추출합니다."""

    
    prompt = f"""사용자의 질문에서 카드 혜택 검색을 위한 핵심 키워드 1~3개를 리스트 형태로 추출해줘.
예: "커피 할인되고 공과금 아끼는 카드" -> ["커피", "공과금"]
질문: {user_query}
형식: ["키워드1", "키워드2"] (JSON 리스트로만 응답하세요)"""
    
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    try:
        return json.loads(response.choices[0].message.content)
    except:
        return [user_query]

# ==========================================
# [CORE] 통합 RAG 함수
# ==========================================
def card_concierge_rag(user_query, top_n=3):
    print(f"🔍 질문 분석 및 의도 추출 중...")
    
    # [Step 1] 동적 의도 추출
    search_intents = extract_search_intents(user_query)
    print(f"🎯 추출된 키워드: {search_intents}")

    # [Step 2] ChromaDB 교차 검색 (AND 로직)
    intent_hits = []
    for intent in search_intents:
        query_text = f"task: search result | query: {intent}"
        results = collection.query(
            query_texts=[query_text],
            n_results=100,
            include=['metadatas']
        )
        ids = {m['card_id'] for m in results['metadatas'][0]}
        intent_hits.append(ids)

    # 교집합 필터링
    common_ids = intent_hits[0]
    for h in intent_hits[1:]:
        common_ids &= h

    if not common_ids:
        print("⚠️ 교집합 결과가 없어 단일 의도 결과로 대체합니다.")
        common_ids = intent_hits[0]

    # [Step 3] Cohere v3.5 리랭킹
    candidate_ids = list(common_ids)
    rerank_docs = []
    for c_id in candidate_ids:
        card = card_master_map[c_id]
        context = f"카드명: {card['name']} | 상세혜택: {' '.join(card['full_details'])}"
        rerank_docs.append(context)

    print(f"🎯 Cohere v3.5 리랭킹 중... (후보: {len(rerank_docs)}개)")
    rerank_response = co.rerank(
        model="rerank-v3.5",
        query=user_query,
        documents=rerank_docs,
        top_n=top_n
    )

# [Step 4] 데이터 복원 및 LLM 답변 생성 (GPT-5-Nano)
    final_context = ""
    for i, hit in enumerate(rerank_response.results):
        best_id = candidate_ids[hit.index]
        card = card_master_map[best_id]
        # 순위 정보와 함께 컨텍스트 구성
        final_context += f"### [Top {i+1}] {card['name']} ({card['corp']})\n"
        final_context += f"- 실적조건: {card['structured']['condition']}\n"
        final_context += f"- 상세 데이터: {' / '.join(card['full_details'])}\n\n"

    # ⭐ 친절한 챗봇 스타일의 범용 프롬프트
    prompt = f"""당신은 전여욱님의 든든한 '금융 비서 Gemma-Bot'입니다. 
제공된 [카드 데이터]를 분석하여 사용자의 질문에 다정하고 명확하게 답변하세요.

사용자 질문: {user_query}

[카드 데이터]
{final_context}

[답변 작성 가이드]
1. **친절한 인사**: 사용자의 니즈를 언급하며 다정하게 대화를 시작하세요.
2. **Top 3 목록**: 추천하는 상위 3개 카드의 이름과 카드사를 순서대로 나열하세요.
3. **Top 1 집중 분석**: 1위로 선정된 카드가 사용자의 질문과 관련된 혜택을 구체적으로 어떻게 제공하는지(수치, 한도 등) 상세히 설명하세요.
4. **실전 팁**: 1위 카드를 쓸 때 주의해야 할 전월 실적이나 제외 항목을 '비서처럼' 조언해 주세요.
5. **톤앤매너**: 표는 절대 사용하지 마세요. 읽기 편한 문장과 불렛 포인트를 사용하고 전체 분량은 적절히 조절하세요."""

    # gpt-5-nano 에러 방지: temperature 파라미터 제외
    response = openai_client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "사용자의 상황을 공감하고 데이터를 기반으로 최적의 솔루션을 제안하는 다정한 금융 상담사입니다."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

# ==========================================
# 실행부
# ==========================================
if __name__ == "__main__":
    user_q = "사회초년생인데 배달 음식을 자주 시켜 먹고 교통비 혜택이 좋은 카드가 있을까?"
    
    result = card_concierge_rag(user_q)
    print("\n✨ [금융 비서 Gemma-Bot 답변]:\n")
    print(result)

The tokenizer you are loading from './models/gemma-300m-4080super-extreme' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
C:\Users\Jeon\AppData\Local\Temp\ipykernel_21324\2386220240.py:53: DeprecationWarning: The class GemmaEmbeddingFunction does not implement __init__. This will be required in a future version.
  embedding_function=GemmaEmbeddingFunction()


🔍 질문 분석 및 의도 추출 중...
🎯 추출된 키워드: ['배달', '교통비']
🎯 Cohere v3.5 리랭킹 중... (후보: 16개)

✨ [금융 비서 Gemma-Bot 답변]:

안녕하세요! 사회초년생으로 배달 음식 자주 시키시고 교통비 혜택까지 생각하신다니, Gemma-Bot이 현명하게 도와드릴게요.

상위 3개 카드 추천
1) 뉴타임카드 (SC제일은행)
2) 토스 USS NEXT 체크카드 (KB국민카드)
3) 라이언 치즈 체크카드 (NH농협카드)

Top 1 집중 분석: 뉴타임카드가 왜 당신에게 잘 맞나
- 배달앱 혜택: 배달의민족, 요기요, 배달통에서 5% 할인. 건당 최소 5,000원 이상 결제 시, 22:00~00:00에 해당 금액에 적용되며 1일 1회 할인. 단, 월간 할인 한도는 전월 실적에 따라 달라집니다.
- 전월 실적에 따른 월간 한도: 30만원 이상 50만원 미만 시 1만원, 50만원 이상 70만원 미만 시 2만원, 70만원 이상 시 3만원까지 한도가 적용됩니다. 즉, 월 실적이 높을수록 더 많은 할인 혜택을 받게 됩니다.
- 교통 혜택(대중교통): All day 서비스로 지하철·버스 이용금액에 5% 할인. 후불교통 기능을 통해 납부하는 경우에 적용되며, 전월 실적 한도 이내에서 할인됩니다.
- 점심/저녁 혜택: 점심(12:00~14:00)과 저녁(19:00~21:00) 시간대에 음식점 5% 할인. 1일 1회, 결제 금액이 최소 5,000원 이상일 때 적용됩니다.
- 편의점/제과점 Time 할인: 07:00~09:00에 편의점 및 제과점 5% 할인(일 1회). 대상은 BC카드 등록 업종 분류에 따라 적용됩니다.
- 기타: 해외 이용 5% 등 다양한 부가 혜택이 있지만, 모두 전월 실적 기반의 월 한도 내에서 적용됩니다.

실전 팁: 1위 카드를 쓸 때 꼭 알아두면 좋은 점
- 전월 실적 관리가 핵심: 월 할인 한도는 전월 실적에 따라 달라지므로, 배달·교통 지출을 한 달에 어느 정도 몰아치면 더 큰 혜택을 받을 수 있습니다. 예를 

In [12]:
def print_all_card_details(target_card_id):
    # 1. 해당 ID에 맞는 모든 조각(fragments) 찾기
    card_fragments = [item for item in master_data if item['metadata']['card_id'] == target_card_id]
    
    if not card_fragments:
        print(f"❌ ID {target_card_id}에 해당하는 카드를 찾을 수 없습니다.")
        return

    # 2. 공통 정보 출력 (첫 번째 조각 기준)
    base_info = card_fragments[0]['metadata']
    print(f"\n{'='*30} CARD FACT CHECK {'='*30}")
    print(f"💳 카드명: {base_info['card_name']} ({base_info['corp']})")
    print(f"🆔 Card ID: {base_info['card_id']}")
    print(f"💰 연회비: {base_info['annual_fee']}")
    print(f"📉 최소실적: {base_info['min_performance']}원")
    print(f"{'='*77}\n")

    # 3. 각 혜택 카테고리별 상세 내용 출력
    for i, fragment in enumerate(card_fragments):
        meta = fragment['metadata']
        ai = fragment['ai_structured']
        
        print(f"[{i+1}] 카테고리: {meta['main_category']} > {meta['sub_category']} ({meta['category']})")
        print(f"   📝 AI 요약: {ai['summary']}")
        print(f"   🏢 주요 가맹점: {', '.join(ai['merchants'])}")
        print(f"   ✅ 상세 조건: {ai['condition']}")
        print(f"   💡 혜택 상세: {ai['benefit_detail']}")
        
        # 실적 제외 항목이나 유의사항이 있다면 추가 출력
        if ai.get('performance_exclusions') and ai['performance_exclusions'][0] != "정보 없음":
            print(f"   ⚠️ 실적 제외: {ai['performance_exclusions']}")
        
        print("-" * 77)

# 실행: 신한카드 Mr.Life(ID 13)의 모든 내용 확인
print_all_card_details(483)


============================== CARD FACT CHECK ==============================
💳 카드명: 뉴타임카드 (SC제일은행)
🆔 Card ID: 483
💰 연회비: 국내전용 [8,000]원 / 해외겸용 [10,000]원
📉 최소실적: 300000원

[1] 카테고리: 쇼핑/생활 > 편의점 (편의점)
   📝 AI 요약: 뉴타임카드는 주요 편의점 및 제과점에서 5% 할인을 제공합니다.
   🏢 주요 가맹점: GS25, CU, 세븐일레븐, 미니스톱, 이마트24, 파리바게트, 뚜레쥬르
   ✅ 상세 조건: 전월 실적에 따른 월간 통합 할인 한도 이내에서 적용
   💡 혜택 상세: 5% 할인
-----------------------------------------------------------------------------
[2] 카테고리: 푸드/카페 > 푸드 (푸드)
   📝 AI 요약: 뉴타임카드는 점심과 저녁 시간대에 음식점에서 5% 할인을 제공합니다.
   🏢 주요 가맹점: 일반한식, 갈비전문점, 한정식, 스낵, 일식회집, 중국음식, 서양음식, 위탁급식업
   ✅ 상세 조건: 전월 실적에 따른 월간 통합 할인 한도 이내에서 적용
   💡 혜택 상세: 음식점 5% 할인, 1일 1회, 최소 5천원 이상 결제 시 제공
-----------------------------------------------------------------------------
[3] 카테고리: 쇼핑/생활 > 온라인쇼핑 (온라인쇼핑)
   📝 AI 요약: 뉴타임카드는 주요 인터넷 쇼핑에서 5% 할인을 제공합니다.
   🏢 주요 가맹점: G마켓, 11번가, 옥션
   ✅ 상세 조건: 전월 실적에 따른 월간 통합 할인 한도 이내에서 적용
   💡 혜택 상세: 5% 할인
-----------------------------------------------------------------------------
[4] 카테고리: 쇼핑/생